In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder,OrdinalEncoder,RobustScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv("student.csv")
df.head()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [ ]:
def performance_level(score):
    if score < 10:
        return "Low"
    elif score < 15:
        return "Medium"
    else:
        return "High"

df["performance"] = df["G3"].apply(performance_level)


In [ ]:
le = LabelEncoder()
for col in df.select_dtypes(include="object").columns:
    df[col] = le.fit_transform(df[col])


In [ ]:
X = df.drop(["G3", "performance"], axis=1)
y = df["performance"]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
model = LogisticRegression(multi_class="multinomial",solver="lbfgs",max_iter=5000)
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=5000, multi_class='multinomial')

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8888888888888888
Confusion Matrix:
 [[20  0  3]
 [ 0 33  3]
 [ 1  4 35]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.87      0.91        23
           1       0.89      0.92      0.90        36
           2       0.85      0.88      0.86        40

    accuracy                           0.89        99
   macro avg       0.90      0.89      0.89        99
weighted avg       0.89      0.89      0.89        99



In [ ]:
num_col=X.select_dtypes(include='number').columns
cat_col=X.select_dtypes(exclude='number').columns

In [ ]:
preprocessing=ColumnTransformer(
    transformers=[
        ('scaler',StandardScaler(),num_col),
        ('onehotencoder',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),cat_col),
    ]
)

In [ ]:
decisiontreepipeline=Pipeline(
    steps=[
        ('preprocessing',preprocessing),
        ('model',DecisionTreeRegressor())
    ]
)
decisiontreepipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index([], dtype='object'))])),
                ('model', DecisionTreeRegressor())])

In [ ]:
print("Train_score:",decisiontreepipeline.score(X_train,y_train))
print("Test_score:",decisiontreepipeline.score(X_test,y_test))

Train_score: 1.0
Test_score: 0.4840282447881641


#IMPUTER

#Churn=Yes(1)->Custom left the telecom service
#Churn=No(0)->Custom continued the telecom service

In [ ]:
df=pd.read_csv("/content/telco_data.csv")
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1.0,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34.0,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2.0,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45.0,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2.0,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24.0,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72.0,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11.0,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,NaN,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            6293 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           6043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            4543 non-null   float64
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   6043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       5543 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
df=df.drop(columns=['customerID'])

def clean_charges(val):
  val=str(val).strip()
  if val=='':
    return None
  return float(val)

df['TotalCharges']=df['TotalCharges'].apply(clean_charges)

In [ ]:
x=df.drop('Churn',axis=1)
y=df['Churn']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)

In [ ]:
num_col=x.select_dtypes(include='number').columns
obj_col=x.select_dtypes(exclude='number').columns

In [ ]:
# df['TotalCharges']=df['TotalCharges'].str.strip()
# df['TotalCharges']=df['TotalCharges'].replace('',None)
# df['TotalCharges'].astype({'TotalCharges':'float'})

###SIMPLE IMPUTER:
* It replace the missing values with help of strategy (mean,median,most_frequent) or constant with fill_value
* It accepts only 2D

In [ ]:
# imputer=SimpleImputer(strategy='most_frequent')
# imputer=SimpleImputer(strategy='constant',fill_value='UnKnown')

In [ ]:
# df[['gender']]=imputer.fit_transform(df[['gender']])

In [ ]:
# df[df['gender'].isna()]
# df[['gender']].value_counts()

In [ ]:
# imputer.transform(df[['Partner']])

In [ ]:
num_preprocessing=Pipeline(
    steps=[
        ('imputer_for_numcols',SimpleImputer(strategy='mean')),
         ('standardscaler',StandardScaler())
    ]
)
num_preprocessing.fit_transform(xtrain[num_col])
# pipeline.fit(obj_col)
# pipeline.transform()

array([[-0.4377492 ,  0.        ,  0.        , -0.42210502],
       [-0.4377492 ,  0.        ,  0.        ,  1.25536015],
       [-0.4377492 ,  0.        ,  0.        , -1.00299144],
       ...,
       [-0.4377492 , -0.93381312, -1.46464784, -0.87799925],
       [ 2.28441306, -0.93381312,  1.15806793, -0.48254445],
       [-0.4377492 , -0.29753207, -1.50986708, -0.81110232]])

In [ ]:
cat_preprocessing=Pipeline(
    steps=[
        ('imputer_for_objcols',SimpleImputer(strategy='constant',fill_value='Unknown')),
         ('ordinalencoder',OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]
)
cat_preprocessing.fit_transform(xtrain[obj_col])

In [ ]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_preprocessing',num_preprocessing,num_col),
         ('cat_preprocessing',cat_preprocessing,obj_col)
    ]
)
preprocessing.fit_transform(xtrain)


array([[-0.4377492 ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  3.        ],
       [-0.4377492 ,  0.        ,  0.        , ...,  2.        ,
         1.        ,  0.        ],
       [-0.4377492 ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  2.        ],
       ...,
       [-0.4377492 , -0.93381312, -1.46464784, ...,  0.        ,
         1.        ,  2.        ],
       [ 2.28441306, -0.93381312,  1.15806793, ...,  0.        ,
         1.        ,  2.        ],
       [-0.4377492 , -0.29753207, -1.50986708, ...,  1.        ,
         0.        ,  1.        ]])

In [ ]:
pipeline=Pipeline(
    steps=[
        ('preprocessing',preprocessing),
        ('model',LogisticRegression())
    ]
)
pipeline.fit(xtrain,ytrain)
pipeline.named_steps['model']

LogisticRegression()

In [ ]:
decisiontreepipeline=Pipeline(
    steps=[
        ('preprocessing',preprocessing),
        ('model',DecisionTreeClassifier())
    ]
)
decisiontreepipeline.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_preprocessing',
                                                  Pipeline(steps=[('imputer_for_numcols',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat_preprocessing',
                                                  Pipeline(steps=[('imputer_for_objcols',
                                                                   SimpleImputer(fill_value='Un...
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('model', DecisionTreeClassifier())])

In [ ]:
print("Train_score:",decisiontreepipeline.score(xtrain,ytrain))
print("Test_score:",decisiontreepipeline.score(xtest,ytest))

Train_score: 0.998757543485978
Test_score: 0.6941092973740242


In [ ]:
grid_search_cv=GridSearchCV(
    estimator=pipeline,
    param_grid={
        'model__C':[0.01,0.1,1.0,10],
        'model__penalty':['l1','l2'],
        'model__solver':['liblinear'],
        'model__class_weight':['balanced']
    },
    cv=10,
    n_jobs=-1,
    verbose=1,
    scoring='f1_macro'
)

In [ ]:
grid_search_cv.fit(xtrain,ytrain)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num_preprocessing',
                                                                         Pipeline(steps=[('imputer_for_numcols',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                        ('cat_preprocessing',
                                                                         Pipeline(steps=[('imputer_for_objcols'...
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                                       ('model', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'model__C': [0.01, 0.1, 1.0, 10],
                         'model__class_weight': ['balanced'],
                         'model__penalty': ['l1', 'l2'],
                         'model__solver': ['liblinear']},
             scoring='f1_macro', verbose=1)

In [ ]:
grid_search_cv.score(xtrain,ytrain)

0.7120140852180937

In [ ]:
grid_search_cv.best_params_

{'model__C': 0.1,
 'model__class_weight': 'balanced',
 'model__penalty': 'l2',
 'model__solver': 'liblinear'}

In [ ]:
model=grid_search_cv.best_estimator_

In [ ]:
model.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_preprocessing',
                                                  Pipeline(steps=[('imputer_for_numcols',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat_preprocessing',
                                                  Pipeline(steps=[('imputer_for_objcols',
                                                                   SimpleImputer(fill_value='Un...
                                                                                  unknown_value=-1))]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('model',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    solver='liblinear'))])

In [ ]:
model.score(xtrain,ytrain)

0.7412140575079872

In [ ]:
grid_search_cv.best_score_

np.float64(0.7088434266097943)

In [ ]:
cv_result=pd.DataFrame(grid_search_cv.cv_results_)

In [ ]:
cv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__C,param_model__class_weight,param_model__penalty,param_model__solver,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.088058,0.005378,0.027888,0.007118,0.01,balanced,l1,liblinear,"{'model__C': 0.01, 'model__class_weight': 'bal...",0.721296,...,0.673920,0.700266,0.717250,0.696636,0.676754,0.727019,0.692522,0.698923,0.018514,8
1,0.087155,0.004150,0.024671,0.001292,0.01,balanced,l2,liblinear,"{'model__C': 0.01, 'model__class_weight': 'bal...",0.728510,...,0.711996,0.703470,0.713267,0.713377,0.679713,0.732067,0.687789,0.705610,0.016436,7
2,0.136360,0.010823,0.025384,0.002943,0.10,balanced,l1,liblinear,"{'model__C': 0.1, 'model__class_weight': 'bala...",0.730555,...,0.707938,0.708102,0.706820,0.711090,0.684627,0.727019,0.690179,0.705749,0.014723,6
3,0.090849,0.003962,0.024782,0.002442,0.10,balanced,l2,liblinear,"{'model__C': 0.1, 'model__class_weight': 'bala...",0.732850,...,0.714380,0.704887,0.710897,0.716613,0.683812,0.722244,0.698934,0.708843,0.013001,1
4,0.152899,0.010973,0.025392,0.004380,1.00,balanced,l1,liblinear,"{'model__C': 1.0, 'model__class_weight': 'bala...",0.727291,...,0.710301,0.712704,0.706067,0.714320,0.686139,0.717366,0.700532,0.707285,0.012902,3
5,0.097615,0.008393,0.023249,0.000678,1.00,balanced,l2,liblinear,"{'model__C': 1.0, 'model__class_weight': 'bala...",0.728922,...,0.710301,0.712704,0.706067,0.714320,0.684560,0.717366,0.700532,0.707448,0.013157,2
6,0.153339,0.009399,0.025800,0.004043,10.00,balanced,l1,liblinear,"{'model__C': 10, 'model__class_weight': 'balan...",0.728922,...,0.710301,0.712704,0.706067,0.716613,0.684560,0.715746,0.698934,0.707040,0.013814,4
7,0.090846,0.002664,0.022307,0.002600,10.00,balanced,l2,liblinear,"{'model__C': 10, 'model__class_weight': 'balan...",0.728922,...,0.710301,0.712704,0.706067,0.716613,0.684560,0.715746,0.698934,0.707040,0.013814,4
